## <b><font color='darkblue'>Callbacks: Observe, Customize, and Control Agent Behavior</font></b>
([source](https://google.github.io/adk-docs/callbacks/))

In [15]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>Introduction: What are Callbacks and Why Use Them?</font></b>
<font size='3ptx'><b>Callbacks are a cornerstone feature of ADK, providing a powerful mechanism to hook into an agent's execution process.</b> They allow you to observe, customize, and even control the agent's behavior at specific, predefined points without modifying the core ADK framework code.</b>

<b><font size='3ptx'>What are they?</font></b>

In essence, callbacks are standard functions that you define. You then associate these functions with an agent when you create it. The ADK framework automatically calls your functions at key stages, letting you observe or intervene. Think of it like checkpoints during the agent's process:
* <b><font size='3ptx'>Before the agent starts its main work on a request</font></b>, and after it finishes: When you ask an agent to do something (<font color='brown'>e.g., answer a question</font>), it runs its internal logic to figure out the response.
* The <b><font size='3ptx'>Before Agent</font></b> callback executes right before this main work begins for that specific request.
* The <b><font size='3ptx'>After Agent</font></b> callback executes right after the agent has finished all its steps for that request and has prepared the final result, but just before the result is returned.
* <b><font size='3ptx'>This "main work" encompasses the agent's entire process for handling that single request</font></b>. This might involve deciding to call an LLM, actually calling the LLM, deciding to use a tool, using the tool, processing the results, and finally putting together the answer. These callbacks essentially wrap the whole sequence from receiving the input to producing the final output for that one interaction.
* <b><font size='3ptx'>Before sending a request to, or after receiving a response from, the Large Language Model (LLM)</font></b>: These callbacks (<b><font size='3ptx'>Before Model</font></b>, <b><font size='3ptx'>After Model</font></b>) allow you to inspect or modify the data going to and coming from the LLM specifically.
* <b><font size='3ptx'>Before executing a tool</font></b> (<font color='brown'>like a Python function or another agent</font>) <b><font size='3ptx'>or after it finishes</font></b>: Similarly, <b><font size='3ptx'>Before Tool</font></b> and <b><font size='3ptx'>After Tool</font></b> callbacks give you control points specifically around the execution of tools invoked by the agent.

![flow](https://google.github.io/adk-docs/assets/callback_flow.png)

<b><font size='3ptx'>Why use them?</font></b> Callbacks unlock significant flexibility and enable advanced agent capabilities:
* <b><font size='3ptx'>Observe & Debug</font></b>: Log detailed information at critical steps for monitoring and troubleshooting.
* <b><font size='3ptx'>Customize & Control</font></b>: Modify data flowing through the agent (<font color='brown'>like LLM requests or tool results</font>) or even bypass certain steps entirely based on your logic.
* <b><font size='3ptx'>Implement Guardrails</font></b>: Enforce safety rules, validate inputs/outputs, or prevent disallowed operations.
* <b><font size='3ptx'>Manage State</font></b>: Read or dynamically update the agent's session state during execution.
* <b><font size='3ptx'>Integrate & Enhance</font></b>: Trigger external actions (<font color='brown'>API calls, notifications</font>) or add features like caching.

<b><font size='3ptx'>How are they added</font></b>:

```python
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from typing import Optional

# --- Define your callback function ---
def my_before_model_logic(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    print(f"Callback running before model call for agent: {callback_context.agent_name}")
    # ... your custom logic here ...
    return None # Allow the model call to proceed

# --- Register it during Agent creation ---
my_agent = LlmAgent(
    name="MyCallbackAgent",
    model="gemini-2.0-flash", # Or your desired model
    instruction="Be helpful.",
    # Other agent parameters...
    before_model_callback=my_before_model_logic # Pass the function here
)
```

### <b><font color='darkgreen'>The Callback Mechanism: Interception and Control</font></b>
([source](https://google.github.io/adk-docs/callbacks/#the-callback-mechanism-interception-and-control)) <font size='3ptx'><b>When the ADK framework encounters a point where a callback can run (<font color='brown'>e.g., just before calling the LLM</font>), it checks if you provided a corresponding callback function for that agent</b>. If you did, the framework executes your function.</font>

<b><font size='3ptx'>Context is Key</font></b>: Your callback function isn't called in isolation. The framework provides special context objects (<font color='brown'>`CallbackContext` or `ToolContext`</font>) as arguments. <b>These objects contain vital information about the current state of the agent's execution, including the invocation details, session state, and potentially references to services like artifacts or memory</b>. You use these context objects to understand the situation and interact with the framework. (<font color='brown'>See the dedicated "Context Objects" section for full details</font>).

<b><font size='3ptx'>Controlling the Flow (The Core Mechanism)</font></b>: The most powerful aspect of callbacks lies in how their return value influences the agent's subsequent actions. This is how you intercept and control the execution flow:

1. <b><font size='3ptx'>`return None` (Allow Default Behavior)</font></b>:
    - This is the standard way to signal that your callback has finished its work (<font color='brown'>e.g., logging, inspection, minor modifications to mutable input arguments like `llm_request`</font>) and that the ADK agent should <b>proceed with its normal operation</b>.
    - For `before_*` callbacks (`before_agent`, `before_model`, `before_tool`), returning None means the next step in the sequence (<font color='brown'>running the agent logic, calling the LLM, executing the tool</font>) will occur.
    - For `after_*` callbacks (`after_agent`, `after_model`, `after_tool`), returning None means the result just produced by the preceding step (<font color='brown'>the agent's output, the LLM's response, the tool's result</font>) will be used as is.
2. <b><font size='3ptx'>`return <Specific Object>` (Override Default Behavior)</font></b>:
    - <b>Returning a specific type of object</b> (<font color='brown'>instead of `None`</font>) <b>is how you override the ADK agent's default behavior</b>. The framework will use the object you return and skip the step that would normally follow or replace the result that was just generated.
    - <b>`before_agent_callback` → `types.Content`</b>: Skips the agent's main execution logic (`_run_async_impl` / `_run_live_impl`). The returned `Content` object is immediately treated as the agent's final output for this turn. <b>Useful for handling simple requests directly or enforcing access control</b>.
    - <b>`before_model_callback` → `LlmResponse`</b>: Skips the call to the external Large Language Model. The returned `LlmResponse` object is processed as if it were the actual response from the LLM. <b>Ideal for implementing input guardrails, prompt validation, or serving cached responses</b>.
    - <b>`before_tool_callback` → `dict`</b>: Skips the execution of the actual tool function (<font color='brown'>or sub-agent</font>). The returned `dict` is used as the result of the tool call, which is then typically passed back to the LLM. <b>Perfect for validating tool arguments, applying policy restrictions, or returning mocked/cached tool results</b>.
    - <b>`after_agent_callback` → `types.Content`</b>: <b>Replaces the `Content` that the agent's run logic just produced</b>.
    - <b>`after_model_callback` → `LlmResponse`</b>: Replaces the `LlmResponse` received from the LLM. <b>Useful for sanitizing outputs, adding standard disclaimers, or modifying the LLM's response structure</b>.
    - <b>`after_tool_callback` → `dict`</b>: Replaces the `dict` result returned by the tool. <b>Allows for post-processing or standardization of tool outputs before they are sent back to the LLM</b>.

<b>Conceptual Code Example (Guardrail)</b>:
This example demonstrates the common pattern for a guardrail using `before_model_callback`.

In [4]:
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from typing import Optional
from google.genai import types 
from google.adk.sessions import InMemorySessionService

GEMINI_2_FLASH="gemini-2.0-flash"
APP_NAME = "guardrail_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

In [2]:
# --- Define the Callback Function ---
def simple_before_model_modifier(
    callback_context: CallbackContext,
    llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Inspects/modifies the LLM request or skips the call."""
    agent_name = callback_context.agent_name
    print(f"[Callback] Before model call for agent: {agent_name}")

    # Inspect the last user message in the request contents
    last_user_message = ""
    if llm_request.contents and llm_request.contents[-1].role == 'user':
      if llm_request.contents[-1].parts:
        last_user_message = llm_request.contents[-1].parts[0].text
    print(f"[Callback] Inspecting last user message: '{last_user_message}'")

    # --- Modification Example ---
    # Add a prefix to the system instruction
    original_instruction = llm_request.config.system_instruction or types.Content(role="system", parts=[])
    prefix = "[Modified by Callback] "
    # Ensure system_instruction is Content and parts list exists
    if not isinstance(original_instruction, types.Content):
      # Handle case where it might be a string (though config expects Content)
      original_instruction = types.Content(role="system", parts=[types.Part(text=str(original_instruction))])
    if not original_instruction.parts:
      original_instruction.parts.append(types.Part(text="")) # Add an empty part if none exist

    # Modify the text of the first part
    modified_text = prefix + (original_instruction.parts[0].text or "")
    original_instruction.parts[0].text = modified_text
    llm_request.config.system_instruction = original_instruction
    print(f"[Callback] Modified system instruction to: '{modified_text}'")

    # --- Skip Example ---
    # Check if the last user message contains "BLOCK"
    if "BLOCK" in last_user_message.upper():
      print("[Callback] 'BLOCK' keyword found. Skipping LLM call.")
      # Return an LlmResponse to skip the actual LLM call
      return LlmResponse(
          content=types.Content(
              role="model",
              parts=[types.Part(text="LLM call was blocked by `before_model_callback`.")],
          )
      )
    else:
      print("[Callback] Proceeding with LLM call.")
      # Return None to allow the (modified) request to go to the LLM
      return None

In [3]:
# Create LlmAgent and Assign Callback
my_llm_agent = LlmAgent(
    name="ModelCallbackAgent",
    model=GEMINI_2_FLASH,
    instruction="You are a helpful assistant.", # Base instruction
    description="An LLM agent demonstrating before_model_callback",
    before_model_callback=simple_before_model_modifier # Assign the function here
)

In [5]:
# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(agent=my_llm_agent, app_name=APP_NAME, session_service=session_service)
  return session, runner

In [6]:
# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  session, runner = await setup_session_and_runner()
  events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)

In [7]:
await call_agent_async("write a joke on BLOCK")

[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'write a joke on BLOCK'
[Callback] Modified system instruction to: '[Modified by Callback] You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_callback"'
[Callback] 'BLOCK' keyword found. Skipping LLM call.
Agent Response:  LLM call was blocked by `before_model_callback`.


By understanding this mechanism of returning `None` versus returning specific objects, you can precisely control the agent's execution path, making callbacks an essential tool for building sophisticated and reliable agents with ADK.

## <b><font color='darkblue'>Types of Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/types-of-callbacks/#types-of-callbacks)) <font size='3ptx'><b>The framework provides different types of callbacks that trigger at various stages of an agent's execution</b>. Understanding when each callback fires and what context it receives is key to using them effectively.</font>

### <b><font color='darkgreen'>Agent Lifecycle Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/types-of-callbacks/#agent-lifecycle-callbacks)) <b><font size='3ptx'>These callbacks are available on any agent that inherits from <font color='blue'>BaseAgent</font></font></b> (<font color='brown'>including `LlmAgent`, `SequentialAgent`, `ParallelAgent`, `LoopAgent`, etc</font>).

#### <b>Before Agent Callback</b>
<font size='3ptx'><b>When</b></font>: Called immediately before the agent's `_run_async_impl` (or `_run_live_impl`) method is executed. It runs after the agent's <font color='blue'><b>InvocationContext</b></font> is created but before its core logic begins.

<font size='3ptx'><b>Purpose</b></font>: Ideal for setting up resources or state needed only for this specific agent's run, performing validation checks on the session state (`callback_context.state`) before execution starts, logging the entry point of the agent's activity, or potentially modifying the invocation context before the core logic uses it.

In [16]:
show_source_code('callback_example1.py')


```python
#!/usr/bin/env python
import asyncio
from collections import namedtuple
from functools import cache

# ADK Imports
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.runners import InMemoryRunner, Runner  # Use InMemoryRunner
from google.adk.sessions import Session
from google.genai import types # For types.Content
from typing import Optional, NamedTuple

# Define the model - Use the specific model name requested
GEMINI_2_FLASH="gemini-2.0-flash"

# Defined the namedtuple
class AgentEnv(NamedTuple):
  app_name: str
  user_id: str
  session_id_run: str
  session_id_skip: str
  runner: Runner


# --- 1. Define the Callback Function ---
def check_if_agent_should_run(callback_context: CallbackContext) -> Optional[types.Content]:
  """
  Logs entry and checks 'skip_llm_agent' in session state.
  If True, returns Content to skip the agent's execution.
  If False or not present, returns None to allow execution.
  """
  agent_name = callback_context.agent_name
  invocation_id = callback_context.invocation_id
  current_state = callback_context.state.to_dict()

  print(f"\n[Callback] Entering agent: {agent_name} (Inv: {invocation_id})")
  print(f"[Callback] Current State: {current_state}")

  # Check the condition in session state dictionary
  if current_state.get("skip_llm_agent", False):
    print(f"[Callback] State condition 'skip_llm_agent=True' met: Skipping agent {agent_name}.")
    # Return Content to skip the agent's run
    return types.Content(
        parts=[types.Part(text=f"Agent {agent_name} skipped by `before_agent_callback` due to state.")],
        role="model" # Assign model role to the overriding response
    )

  print(f"[Callback] State condition not met: Proceeding with agent {agent_name}.")
  # Return None to allow the LlmAgent's normal execution
  return None


# --- 2. Setup Agent with Callback ---
llm_agent_with_before_cb = LlmAgent(
    name="MyControlledAgent",
    model=GEMINI_2_FLASH,
    instruction="You are a concise assistant.",
    description="An LLM agent demonstrating stateful before_agent_callback",
    before_agent_callback=check_if_agent_should_run # Assign the callback
)


@cache
async def initialize_session_and_runner():
  app_name = "before_agent_demo"
  user_id = "test_user"
  session_id_run = "session_will_run"
  session_id_skip = "session_will_skip"

  # Use InMemoryRunner - it includes InMemorySessionService
  runner = InMemoryRunner(agent=llm_agent_with_before_cb, app_name=app_name)
  # Get the bundled session service to create sessions
  session_service = runner.session_service

  # Create session 1: Agent will run (default empty state)
  await session_service.create_session(
      app_name=app_name,
      user_id=user_id,
      session_id=session_id_run
      # No initial state means 'skip_llm_agent' will be False in the callback check
  )

  # Create session 2: Agent will be skipped (state has skip_llm_agent=True)
  await session_service.create_session(
      app_name=app_name,
      user_id=user_id,
      session_id=session_id_skip,
      state={"skip_llm_agent": True} # Set the state flag here
  )

  return AgentEnv(
      app_name=app_name,
      user_id=user_id,
      session_id_run=session_id_run,
      session_id_skip=session_id_skip,
      runner=runner)


async def query(text: str, agent_env, session_id):
  async for event in agent_env.runner.run_async(
      user_id=agent_env.user_id,
      session_id=session_id,
      new_message=types.Content(
          role="user", parts=[types.Part(text=text)])
  ):
    # Print final output (either from LLM or callback override)
    if event.is_final_response() and event.content:
      print(f"Final Output: [{event.author}] {event.content.parts[0].text.strip()}")
    elif event.is_error():
      print(f"Error Event: {event.error_details}")


async def main():
  agent_env = await initialize_session_and_runner()

  # --- Scenario 1: Run where callback allows agent execution ---
  await query(
      'Hello, please respond.',
      agent_env,
      agent_env.session_id_run)

  # --- Scenario 2: Run where callback intercepts and skips agent ---
  await query(
      "This message won't reach the LLM.",
      agent_env,
      agent_env.session_id_skip)


if __name__ == '__main__':
  asyncio.run(main())
```

In [18]:
import callback_example1

agent_env = await callback_example1.initialize_session_and_runner()

In [24]:
# --- Scenario 1: Run where callback allows agent execution ---
await callback_example1.query(
    'Hello, please respond.',
    agent_env,
    agent_env.session_id_run)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



[Callback] Entering agent: MyControlledAgent (Inv: e-936f9530-2eef-4ae4-b22d-d630e0f433b6)
[Callback] Current State: {}
[Callback] State condition not met: Proceeding with agent MyControlledAgent.
Final Output: [MyControlledAgent] Hello! How can I assist you?


In [22]:
# --- Scenario 2: Run where callback intercepts and skips agent ---
await callback_example1.query(
    "This message won't reach the LLM.",
    agent_env,
    agent_env.session_id_skip)


[Callback] Entering agent: MyControlledAgent (Inv: e-09e4f747-403c-4fe2-aaae-626206898b49)
[Callback] Current State: {'skip_llm_agent': True}
[Callback] State condition 'skip_llm_agent=True' met: Skipping agent MyControlledAgent.
Final Output: [MyControlledAgent] Agent MyControlledAgent skipped by `before_agent_callback` due to state.


<b>Note on the `before_agent_callback` Example</b>:
* <b><font size='3ptx'>What it Shows</font></b>: This example demonstrates the `before_agent_callback`. This callback runs right before the agent's main processing logic starts for a given request.
* <b><font size='3ptx'>How it Works</font></b>: The callback function (`check_if_agent_should_run`) looks at a flag (`skip_llm_agent`) in the session's state.
    - If the flag is True, the callback returns a <b><font color='blue'>types.Content</font></b> object. This tells the ADK framework to skip the agent's main execution entirely and use the callback's returned content as the final response.
    - If the flag is False (or not set), the callback returns `None` or an empty object. This tells the ADK framework to proceed with the agent's normal execution (<font color='brown'>calling the LLM in this case</font>).
* <b><font size='3ptx'>Expected Outcome</font></b>: You'll see two scenarios:
    1. In the session with the `skip_llm_agent: True` state, the agent's LLM call is bypassed, and the output comes directly from the callback ("Agent... skipped...").
    2. In the session without that state flag, the callback allows the agent to run, and you see the actual response from the LLM (e.g., "Hello!").
* <b><font size='3ptx'>Understanding Callbacks</font></b>: This highlights how `before_` callbacks act as gatekeepers, allowing you to intercept execution before a major step and potentially prevent it based on checks (<font color='brown'>like state, input validation, permissions</font>).

#### <b>After Agent Callback</b>
<b><font size='3ptx'>When</font></b>: Called immediately after the agent's `_run_async_impl` (or `_run_live_impl`) method successfully completes. It does not run if the agent was skipped due to `before_agent_callback` returning content or if `end_invocation` was set during the agent's run.

<b><font size='3ptx'>Purpose</font></b>: Useful for cleanup tasks, post-execution validation, logging the completion of an agent's activity, modifying final state, or augmenting/replacing the agent's final output.

In [25]:
show_source_code('callback_example2.py')


```python
#!/usr/bin/env python
import asyncio
from functools import cache
import time

# ADK Imports
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.runners import InMemoryRunner, Runner  # Use InMemoryRunner
from google.genai import types # For types.Content
from typing import Optional, NamedTuple

# Define the model - Use the specific model name requested
GEMINI_2_FLASH="gemini-2.0-flash"


# Defined the namedtuple
class AgentEnv(NamedTuple):
  app_name: str
  user_id: str
  session_id_normal: str
  session_id_modify: str
  runner: Runner


# --- 1. Define the Callback Function ---
def modify_output_after_agent(callback_context: CallbackContext) -> Optional[types.Content]:
  """
  Logs exit from an agent and checks 'add_concluding_note' in session state.
  If True, returns new Content to *replace* the agent's original output.
  If False or not present, returns None, allowing the agent's original output to be used.
  """
  agent_name = callback_context.agent_name
  invocation_id = callback_context.invocation_id
  current_state = callback_context.state.to_dict()

  print(f"\n[Callback] Exiting agent: {agent_name} (Inv: {invocation_id})")
  print(f"[Callback] Current State: {current_state}")

  # Example: Check state to decide whether to modify the final output
  if current_state.get("add_concluding_note", False):
    print(f"[Callback] State condition 'add_concluding_note=True' met: Replacing agent {agent_name}'s output.")
    # Return Content to *replace* the agent's own output
    return types.Content(
        parts=[types.Part(text=f"Concluding note added by `after_agent_callback`, replacing original output.")],
        role="model" # Assign model role to the overriding response
    )
  else:
    print(f"[Callback] State condition not met: Using agent {agent_name}'s original output.")
    # Return None - the agent's output produced just before this callback will be used.
    return None


# --- 2. Setup Agent with Callback ---
llm_agent_with_after_cb = LlmAgent(
    name="MySimpleAgentWithAfter",
    model=GEMINI_2_FLASH,
    instruction="You are a simple agent. Just say 'Processing complete!'",
    description="An LLM agent demonstrating after_agent_callback for output modification",
    after_agent_callback=modify_output_after_agent # Assign the callback here
)


@cache
async def initialize_session_and_runner():
  app_name = "before_agent_demo"
  user_id = "test_user"
  session_id_normal = "session_run_normally"
  session_id_modify = "session_modify_output"

  # Use InMemoryRunner - it includes InMemorySessionService
  runner = InMemoryRunner(agent=llm_agent_with_after_cb, app_name=app_name)
  # Get the bundled session service to create sessions
  session_service = runner.session_service

  # Create session 1: Agent will run (default empty state)
  await session_service.create_session(
      app_name=app_name,
      user_id=user_id,
      session_id=session_id_normal
      # No initial state means 'skip_llm_agent' will be False in the callback check
  )

  # Create session 2: Agent will be skipped (state has skip_llm_agent=True)
  await session_service.create_session(
      app_name=app_name,
      user_id=user_id,
      session_id=session_id_modify,
      state={"add_concluding_note": True} # Set the state flag here
  )

  return AgentEnv(
      app_name=app_name,
      user_id=user_id,
      session_id_normal=session_id_normal,
      session_id_modify=session_id_modify,
      runner=runner)


async def query(text: str, agent_env, session_id):
  start_time = time.time()
  async for event in agent_env.runner.run_async(
      user_id=agent_env.user_id,
      session_id=session_id,
      new_message=types.Content(
          role="user", parts=[types.Part(text=text)])
  ):
    # Print final output (either from LLM or callback override)
    if event.is_final_response() and event.content:
      print(f"Final Output: [{event.author}] {event.content.parts[0].text.strip()}")
    elif event.is_error():
      print(f"Error Event: {event.error_details}")

  spent_time_sec = time.time() - start_time
  print(f'[Query] Spent time {spent_time_sec:.02f}s!')


async def main():
  agent_env = await initialize_session_and_runner()

  # --- Scenario 1: Run where callback allows agent's original output ---
  print(
      "\n" + "="*20 +
      f" SCENARIO 1: Running Agent on Session '{agent_env.session_id_normal}' " +
      "(Should Use Original Output) " + "="*20)
  await query(
      'Process this please.',
      agent_env,
      agent_env.session_id_normal)

  # --- Scenario 2: Run where callback replaces the agent's output ---
  print(
      "\n" + "="*20 +
      f" SCENARIO 2: Running Agent on Session '{agent_env.session_id_modify}' " +
      "(Should Replace Output) " + "="*20)
  await query(
      'Process this and add note.',
      agent_env,
      agent_env.session_id_modify)


if __name__ == '__main__':
  asyncio.run(main())
```

In [27]:
import callback_example2

agent_env = await callback_example2.initialize_session_and_runner()

In [29]:
# --- Scenario 1: Run where callback allows agent's original output ---
await callback_example2.query(
      'Process this please.',
      agent_env,
      agent_env.session_id_normal)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Final Output: [MySimpleAgentWithAfter] Processing complete!

[Callback] Exiting agent: MySimpleAgentWithAfter (Inv: e-0c208c70-6670-4c2e-90e3-f0072734a621)
[Callback] Current State: {}
[Callback] State condition not met: Using agent MySimpleAgentWithAfter's original output.
[Query] Spent time 1.60s!


In [30]:
# --- Scenario 2: Run where callback replaces the agent's output ---
await callback_example2.query(
      'Process this and add note.',
      agent_env,
      agent_env.session_id_modify)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Final Output: [MySimpleAgentWithAfter] Processing complete!

[Callback] Exiting agent: MySimpleAgentWithAfter (Inv: e-b0213bf2-c143-47fb-943f-c905627137ce)
[Callback] Current State: {'add_concluding_note': True}
[Callback] State condition 'add_concluding_note=True' met: Replacing agent MySimpleAgentWithAfter's output.
Final Output: [MySimpleAgentWithAfter] Concluding note added by `after_agent_callback`, replacing original output.
[Query] Spent time 1.52s!


<b>Note on the after_agent_callback Example:</b>
* <b><font size='3ptx'>What it Shows</font></b>: This example demonstrates the `after_agent_callback`. This callback runs right after the agent's main processing logic has finished and produced its result, but before that result is finalized and returned.
* <b><font size='3ptx'>How it Works</font></b>: The callback function (`modify_output_after_agent`) checks a flag (`add_concluding_note`) in the session's state.
* <b><font size='3ptx'>Expected Outcome</font></b>: You'll see two scenarios:
    1. In the session without the `add_concluding_note`: True state, the callback allows the agent's original output ("Processing complete!") to be used.
    2. In the session with that state flag, the callback intercepts the agent's original output and replaces it with its own message ("Concluding note added...").
* <b><font size='3ptx'>Understanding Callbacks</font></b>: This highlights how `after_` callbacks allow post-processing or modification. You can inspect the result of a step (<font color='brown'>the agent's run</font>) and decide whether to let it pass through, change it, or completely replace it based on your logic.

### <b><font color='darkgreen'>LLM Interaction Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/types-of-callbacks/#llm-interaction-callbacks)) <font size='3ptx'><b>These callbacks are specific to `LlmAgent` and provide hooks around the interaction with the Large Language Model.</b></font>

#### <b>Before Model Callback</b>
<b><font size='3ptx'>When</font></b>: Called just before the `generate_content_async` (<font color='brown'>or equivalent</font>) request is sent to the LLM within an `LlmAgent`'s flow.

<b><font size='3ptx'>Purpose</font></b>: Allows inspection and modification of the request going to the LLM. Use cases include adding dynamic instructions, injecting few-shot examples based on state, modifying model config, implementing guardrails (<font color='brown'>like profanity filters</font>), or implementing request-level caching.

<b><font size='3ptx'>Return Value Effect</font></b>:
If the callback returns `None`, the LLM continues its normal workflow. If the callback returns an <b><font color='blue'>LlmResponse</font></b> object, then the call to the LLM is skipped. The returned <b><font color='blue'>LlmResponse</font></b> is used directly as if it came from the model. This is powerful for implementing guardrails or caching.

In [31]:
show_source_code('callback_example3.py')


```python
#!/usr/bin/env python
import asyncio
from collections import namedtuple
from functools import cache
import time

from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import Session
from typing import Optional, NamedTuple
from google.genai import types
from google.adk.sessions import InMemorySessionService


GEMINI_2_FLASH="gemini-2.0-flash"
APP_NAME = "guardrail_app"
USER_ID = "user_1"
SESSION_ID = "session_001"


# Defined the namedtuple
class AgentEnv(NamedTuple):
  runner: Runner
  session: Session


# --- Define the Callback Function ---
def simple_before_model_modifier(
    callback_context: CallbackContext,
    llm_request: LlmRequest) -> Optional[LlmResponse]:
  """Inspects/modifies the LLM request or skips the call."""
  agent_name = callback_context.agent_name
  print(f"[Callback] Before model call for agent: {agent_name}")

  # Inspect the last user message in the request contents
  last_user_message = ""
  if llm_request.contents and llm_request.contents[-1].role == 'user':
    if llm_request.contents[-1].parts:
      last_user_message = llm_request.contents[-1].parts[0].text
  print(f"[Callback] Inspecting last user message: '{last_user_message}'")

  # --- Modification Example ---
  # Add a prefix to the system instruction
  original_instruction = llm_request.config.system_instruction or types.Content(role="system", parts=[])
  postfix = " If the query is a math problem. Respond with 'I do not know.'."
  # Ensure system_instruction is Content and parts list exists
  if not isinstance(original_instruction, types.Content):
    # Handle case where it might be a string (though config expects Content)
    original_instruction = types.Content(
        role="system", parts=[types.Part(text=str(original_instruction))])
  if not original_instruction.parts:
    original_instruction.parts.append(types.Part(text="")) # Add an empty part if none exist

  # Modify the text of the first part
  modified_text = (original_instruction.parts[0].text or "") + postfix
  original_instruction.parts[0].text = modified_text
  llm_request.config.system_instruction = original_instruction
  print(f"[Callback] Modified system instruction to: '{modified_text}'")

  # --- Skip Example ---
  # Check if the last user message contains "BLOCK"
  if "BLOCK" in last_user_message.upper():
    print("[Callback] 'BLOCK' keyword found. Skipping LLM call.")
    # Return an LlmResponse to skip the actual LLM call
    return LlmResponse(
        content=types.Content(
            role="model",
            parts=[types.Part(text="LLM call was blocked by before_model_callback.")],
        )
    )
  else:
    print("[Callback] Proceeding with LLM call.")
    # Return None to allow the (modified) request to go to the LLM
    return None


# Create LlmAgent and Assign Callback
my_llm_agent = LlmAgent(
    name="ModelCallbackAgent",
    model=GEMINI_2_FLASH,
    instruction="You are a helpful assistant.", # Base instruction
    description="An LLM agent demonstrating before_model_callback",
    before_model_callback=simple_before_model_modifier, # Assign the function here
)


# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(agent=my_llm_agent, app_name=APP_NAME, session_service=session_service)
  return AgentEnv(runner=runner, session=session)


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  agent_env = await setup_session_and_runner()
  events = agent_env.runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


async def main():
  # --- Scenario 1: Modify system message
  await call_agent_async("I am John. What is my name?")
  await call_agent_async("How much is 1 + 1?")

  # --- Scenario 2: Blocked LLM call
  await call_agent_async("write a joke on BLOCK")


if __name__ == '__main__':
  asyncio.run(main())
```

In [32]:
import callback_example3

In [34]:
# --- Scenario 1.1: Modify system message and process question normally
await callback_example3.call_agent_async("I am John. What is my name?")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'I am John. What is my name?'
[Callback] Modified system instruction to: 'You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_callback" If the query is a math problem. Respond with 'I do not know.'.'
[Callback] Proceeding with LLM call.
Agent Response:  Your name is John.



In [36]:
# --- Scenario 1.2: Modify system message and response will be impacted by the postfix
#                   1 + 1 should be 2. But `I do not know` is forced to be responded.
await callback_example3.call_agent_async("How much is 1 + 1?")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'How much is 1 + 1?'
[Callback] Modified system instruction to: 'You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_callback" If the query is a math problem. Respond with 'I do not know.'.'
[Callback] Proceeding with LLM call.
Agent Response:  I do not know.



In [37]:
# --- Scenario 2: Blocked LLM call
await call_agent_async("Write a joke on BLOCK")

[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'Write a joke on BLOCK'
[Callback] Modified system instruction to: '[Modified by Callback] You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_callback"'
[Callback] 'BLOCK' keyword found. Skipping LLM call.
Agent Response:  LLM call was blocked by `before_model_callback`.


#### <b>After Model Callback</b>
<b><font size='3ptx'>When</font></b>: Called just after a response (`LlmResponse`) is received from the LLM, before it's processed further by the invoking agent.

<b><font size='3ptx'>Purpose</font></b>: Allows inspection or modification of the raw LLM response. Use cases include
- logging model outputs,
- reformatting responses,
- censoring sensitive information generated by the model,
- parsing structured data from the LLM response and storing it in callback_context.state
- or handling specific error codes.

In [38]:
show_source_code('callback_example4.py')


```python
#!/usr/bin/env python
import asyncio
from collections import namedtuple
import copy
from functools import cache
import time

from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.runners import Runner
from google.adk.sessions import Session
from typing import Optional, NamedTuple
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.models import LlmResponse


GEMINI_2_FLASH="gemini-2.0-flash"
APP_NAME = "guardrail_app"
USER_ID = "user_1"
SESSION_ID = "session_001"


# Defined the namedtuple
class AgentEnv(NamedTuple):
  runner: Runner
  session: Session


# --- Define the Callback Function ---
def simple_after_model_modifier(
    callback_context: CallbackContext,
    llm_response: LlmResponse) -> Optional[LlmResponse]:
  """Inspects/modifies the LLM response after it's received."""
  agent_name = callback_context.agent_name
  print(f"[Callback] After model call for agent: {agent_name}")

  # --- Inspection ---
  original_text = ""
  if llm_response.content and llm_response.content.parts:
    # Assuming simple text response for this example
    if llm_response.content.parts[0].text:
      original_text = llm_response.content.parts[0].text
      print(
          "[Callback] Inspected original response text: "
          f"'{original_text[:100]}'...") # Log snippet
    elif llm_response.content.parts[0].function_call:
      print(
          "[Callback] Inspected response: Contains function call "
          "'{llm_response.content.parts[0].function_call.name}'. No text modification.")
      return None  # Don't modify tool calls in this example
    else:
      print("[Callback] Inspected response: No text content found.")
      return None
  elif llm_response.error_message:
    print(
        "[Callback] Inspected response: Contains error "
        f"'{llm_response.error_message}'. No modification.")
    return None
  else:
    print("[Callback] Inspected response: Empty LlmResponse.")
    return None # Nothing to modify

  # --- Modification Example ---
  # Replace "joke" with "funny story" (case-insensitive)
  search_term = "joke"
  replace_term = "funny story"
  if search_term in original_text.lower():
    print(f"[Callback] Found '{search_term}'. Modifying response.")
    modified_text = original_text.replace(search_term, replace_term)
    modified_text = modified_text.replace(
        search_term.capitalize(), replace_term.capitalize())  # Handle capitalization

    # Create a NEW LlmResponse with the modified content
    # Deep copy parts to avoid modifying original if other callbacks exist
    modified_parts = [copy.deepcopy(part) for part in llm_response.content.parts]
    modified_parts[0].text = modified_text # Update the text in the copied part

    new_response = LlmResponse(
        content=types.Content(role="model", parts=modified_parts),
        # Copy other relevant fields if necessary, e.g., grounding_metadata
        grounding_metadata=llm_response.grounding_metadata)
    print(f"[Callback] Returning modified response.")
    return new_response # Return the modified response
  else:
    print(f"[Callback] '{search_term}' not found. Passing original response through.")
    # Return None to use the original llm_response
    return None


# Create LlmAgent and Assign Callback
my_llm_agent = LlmAgent(
    name="AfterModelCallbackAgent",
    model=GEMINI_2_FLASH,
    instruction="You are a helpful assistant.",
    description="An LLM agent demonstrating after_model_callback",
    after_model_callback=simple_after_model_modifier # Assign the function here
)


# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=my_llm_agent, app_name=APP_NAME, session_service=session_service)
  return AgentEnv(runner=runner, session=session)


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  agent_env = await setup_session_and_runner()
  events = agent_env.runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


async def main():
  await call_agent_async('Write two time the word "joke"')


if __name__ == '__main__':
  asyncio.run(main())
```

In [39]:
import callback_example4

await callback_example4.call_agent_async('Write two time the word "joke"')

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[Callback] After model call for agent: AfterModelCallbackAgent
[Callback] Inspected original response text: 'joke joke
'...
[Callback] Found 'joke'. Modifying response.
[Callback] Returning modified response.
Agent Response:  funny story funny story



### <b><font color='darkgreen'>Tool Execution Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/types-of-callbacks/#tool-execution-callbacks)) <font size='3ptx'><b>These callbacks are also specific to `LlmAgent` and trigger around the execution of tools (<font color='brown'>including `FunctionTool`, `AgentTool`, etc.</font>) that the LLM might request.</b></font>

#### <b>Before Tool Callback</b>
<b><font size='3ptx'>When</font></b>: Called just before a specific tool's `run_async` method is invoked, after the LLM has generated a function call for it.

<b><font size='3ptx'>Purpose</font></b>: Allows inspection and modification of tool arguments, performing authorization checks before execution, logging tool usage attempts, or implementing tool-level caching.

<b><font size='3ptx'>Return Value Effect</font></b>:
1. If the callback returns `None`, the tool's `run_async` method is executed with the (<font color='brown'>potentially modified</font>) args.
2. If a dictionary is returned, the tool's `run_async` method is skipped. The returned dictionary is used directly as the result of the tool call. This is useful for caching or overriding tool behavior.

In [40]:
show_source_code('callback_example5.py')


```python
#!/usr/bin/env python
import asyncio
from collections import namedtuple
from functools import cache
import time

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from typing import Optional
from google.genai import types
from google.adk.sessions import InMemorySessionService, Session
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.base_tool import BaseTool
from typing import Dict, Any, NamedTuple


GEMINI_2_FLASH="gemini-2.0-flash"
APP_NAME = "guardrail_app"
USER_ID = "user_1"
SESSION_ID = "session_001"


# Defined the namedtuple
class AgentEnv(NamedTuple):
  runner: Runner
  session: Session


def get_capital_city(country: str) -> str:
  """Retrieves the capital city of a given country."""
  print(f"--- Tool 'get_capital_city' executing with country: {country} ---")
  country_capitals = {
      "united states": "Washington, D.C.",
      "canada": "Ottawa",
      "france": "Paris",
      "germany": "Berlin",
      "taiwan": "taipei",
  }
  return country_capitals.get(
      country.lower(), f"Capital not found for {country}")


capital_tool = FunctionTool(func=get_capital_city)


def simple_before_tool_modifier(
    tool: BaseTool, args: Dict[str, Any],
    tool_context: ToolContext) -> Optional[Dict]:
  """Inspects/modifies tool args or skips the tool call."""
  agent_name = tool_context.agent_name
  tool_name = tool.name
  print(f"[Callback] Before tool call for tool '{tool_name}' in agent '{agent_name}'")
  print(f"[Callback] Original args: {args}")

  if tool_name == 'get_capital_city' and args.get('country', '').lower() == 'canada':
    print("[Callback] Detected 'Canada'. Modifying args to 'France'.")
    args['country'] = 'France'
    print(f"[Callback] Modified args: {args}")
    return None

  # If the tool is 'get_capital_city' and country is 'BLOCK'
  if tool_name == 'get_capital_city' and args.get('country', '').upper() == 'BLOCK':
    print("[Callback] Detected 'BLOCK'. Skipping tool execution.")
    return {"result": "Tool execution was blocked by `before_tool_callback`."}

  print("[Callback] Proceeding with original or previously modified args.")
  return None


my_llm_agent = LlmAgent(
    name="ToolCallbackAgent",
    model=GEMINI_2_FLASH,
    instruction="You are an agent that can find capital cities. Use the get_capital_city tool.",
    description="An LLM agent demonstrating before_tool_callback",
    tools=[capital_tool],
    before_tool_callback=simple_before_tool_modifier)


# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=my_llm_agent,
      app_name=APP_NAME,
      session_service=session_service)
  return AgentEnv(
      session=session,
      runner=runner)


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  agent_env = await setup_session_and_runner()
  events = agent_env.runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  asyncio.run(call_agent_async("Canada"))
```

In [42]:
import callback_example5

await callback_example5.call_agent_async("Canada")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[Callback] Before tool call for tool 'get_capital_city' in agent 'ToolCallbackAgent'
[Callback] Original args: {'country': 'Canada'}
[Callback] Detected 'Canada'. Modifying args to 'France'.
[Callback] Modified args: {'country': 'France'}
--- Tool 'get_capital_city' executing with country: France ---
Agent Response:  I was asked about Canada, but the tool request was for France.  The capital of Canada is Ottawa.



#### <b>After Tool Callback</b>
<b><font size='3ptx'>When</font></b>: Called just after the tool's `run_async` method completes successfully.

<b><font size='3ptx'>Purpose</font></b>: Allows inspection and modification of the tool's result before it's sent back to the LLM (<font color='brown'>potentially after summarization</font>). Useful for logging tool results, post-processing or formatting results, or saving specific parts of the result to the session state.

<b><font size='3ptx'>Return Value Effect</font></b>:
1. If the callback returns `None`, the original `tool_response` is used.
2. If a new dictionary is returned, it replaces the original `tool_response`. This allows modifying or filtering the result seen by the LLM.

In [43]:
show_source_code('callback_example6.py')


```python
#!/usr/bin/env python
import asyncio
from collections import namedtuple
from copy import deepcopy
from functools import cache
import time

from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from typing import Any, Dict, Optional, NamedTuple
from google.genai import types
from google.adk.models import LlmResponse
from google.adk.tools.base_tool import BaseTool


GEMINI_2_FLASH="gemini-2.0-flash"
APP_NAME = "guardrail_app"
USER_ID = "user_1"
SESSION_ID = "session_001"


# Defined the namedtuple
class AgentEnv(NamedTuple):
  runner: Runner
  session: Session


# --- Define a Simple Tool Function (Same as before) ---
def get_capital_city(country: str) -> Dict[str, Any]:
  """Retrieves the capital city of a given country."""
  print(f"--- Tool 'get_capital_city' executing with country: {country} ---")
  country_capitals = {
      "united states": "Washington, D.C.",
      "canada": "Ottawa",
      "france": "Paris",
      "germany": "Berlin",
      "taiwan": "taipei",
  }

  return {
      "result": country_capitals.get(country.lower(), f"Capital not found for {country}")}


# --- Wrap the function into a Tool ---
capital_tool = FunctionTool(func=get_capital_city)


# --- Define the Callback Function ---
def simple_after_tool_modifier(
    tool: BaseTool,
    args: Dict[str, Any],
    tool_context: ToolContext,
    tool_response: Dict
) -> Optional[Dict]:
  """Inspects/modifies the tool result after execution."""
  agent_name = tool_context.agent_name
  tool_name = tool.name
  print(f"[Callback] After tool call for tool '{tool_name}' in agent '{agent_name}'")
  print(f"[Callback] Args used: {args}")
  print(f"[Callback] Original tool_response: {tool_response}")

  # Default structure for function tool results is {"result": <return_value>}
  original_result_value = tool_response.get("result", "")
  # original_result_value = tool_response

  # --- Modification Example ---
  # If the tool was 'get_capital_city' and result is 'Washington, D.C.'
  if tool_name == 'get_capital_city' and original_result_value == "Washington, D.C.":
    print("[Callback] Detected 'Washington, D.C.'. Modifying tool response.")

    # IMPORTANT: Create a new dictionary or modify a copy
    modified_response = deepcopy(tool_response)
    modified_response["result"] = f"{original_result_value} (Note: This is the capital of the USA)."
    modified_response["note_added_by_callback"] = True # Add extra info if needed

    print(f"[Callback] Modified tool_response: {modified_response}")
    return modified_response # Return the modified dictionary

  print("[Callback] Passing original tool response through.")
  # Return None to use the original tool_response
  return None


# Create LlmAgent and Assign Callback
my_llm_agent = LlmAgent(
    name="AfterToolCallbackAgent",
    model=GEMINI_2_FLASH,
    instruction="You are an agent that finds capital cities using the get_capital_city tool. Report the result clearly.",
    description="An LLM agent demonstrating after_tool_callback",
    tools=[capital_tool], # Add the tool
    after_tool_callback=simple_after_tool_modifier, # Assign the callback
)


# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=my_llm_agent,
      app_name=APP_NAME,
      session_service=session_service)
  return AgentEnv(
      session=session,
      runner=runner)


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  agent_env = await setup_session_and_runner()
  events = agent_env.runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  asyncio.run(call_agent_async("united states"))
```

In [45]:
import callback_example6


await callback_example6.call_agent_async("united states")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


--- Tool 'get_capital_city' executing with country: united states ---
[Callback] After tool call for tool 'get_capital_city' in agent 'AfterToolCallbackAgent'
[Callback] Args used: {'country': 'united states'}
[Callback] Original tool_response: {'result': 'Washington, D.C.'}
[Callback] Detected 'Washington, D.C.'. Modifying tool response.
[Callback] Modified tool_response: {'result': 'Washington, D.C. (Note: This is the capital of the USA).', 'note_added_by_callback': True}
Agent Response:  The capital city of the United States is Washington, D.C. (Note: This is the capital of the USA.).



## <b><font color='darkblue'>Design Patterns and Best Practices for Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/design-patterns-and-best-practices/#design-patterns-and-best-practices-for-callbacks)) <font size='3ptx'><b>Callbacks offer powerful hooks into the agent lifecycle. </b>Here are common design patterns illustrating how to leverage them effectively in ADK, followed by best practices for implementation.</font>

### <b><font color='darkgreen'>Design Patterns</font></b>
<b><font size='3ptx'>These patterns demonstrate typical ways to enhance or control agent behavior using callbacks:</font></b>

#### <b>1. Guardrails & Policy Enforcement</b>
* <b><font size='3ptx'>Pattern</font></b>: Intercept requests before they reach the LLM or tools to enforce rules.
* <b><font size='3ptx'>How</font></b>: Use `before_model_callback` to inspect the `LlmRequest` prompt or `before_tool_callback` to inspect tool arguments. If a policy violation is detected (<font color='brown'>e.g., forbidden topics, profanity</font>), return a predefined response (`LlmResponse` or `dict`) to block the operation and optionally update `context.state` to log the violation.
* <b><font size='3ptx'>Example</font></b>: A `before_model_callback` checks `llm_request.contents` for sensitive keywords and returns a standard "Cannot process this request" `LlmResponse` if found, preventing the LLM call.

#### <b>2. Dynamic State Management</b>
* <b><font size='3ptx'>Pattern</font></b>: Read from and write to session state within callbacks to make agent behavior context-aware and pass data between steps.
* <b><font size='3ptx'>How</font></b>: Access `callback_context.state` or `tool_context.state`. Modifications (`state['key'] = value`) are automatically tracked in the subsequent `Event.actions.state_delta` for persistence by the <b><font color='blue'>SessionService</font></b>.
* <b><font size='3ptx'>Example</font></b>: An `after_tool_callback` saves a `transaction_id` from the tool's result to `tool_context.state['last_transaction_id']`. A later `before_agent_callback` might read `state['user_tier']` to customize the agent's greeting.

#### <b>3. Logging and Monitoring</b>
* <b><font size='3ptx'>Pattern</font></b>: Add detailed logging at specific lifecycle points for observability and debugging.
* <b><font size='3ptx'>How</font></b>: Implement callbacks (e.g., `before_agent_callback`, `after_tool_callback`, `after_model_callback`) to print or send structured logs containing information like agent name, tool name, invocation ID, and relevant data from the context or arguments.
* <b><font size='3ptx'>Example</font></b>: Log messages like `INFO: [Invocation: e-123] Before Tool: search_api - Args: {'query': 'ADK'}`.

#### <b>4. Caching</b>
* <b><font size='3ptx'>Pattern</font></b>: Avoid redundant LLM calls or tool executions by caching results.
* <b><font size='3ptx'>How</font></b>: In `before_model_callback` or `before_tool_callback`, generate a cache key based on the request/arguments. Check `context.state` (<font color='brown'>or an external cache</font>) for this key. If found, return the cached `LlmResponse` or result directly, skipping the actual operation. If not found, allow the operation to proceed and use the corresponding `after_` callback (`after_model_callback`, `after_tool_callback`) to store the new result in the cache using the key.
* <b><font size='3ptx'>Example</font></b>: `before_tool_callback` for `get_stock_price(symbol)` checks `state[f"cache:stock:{symbol}"]`. If present, returns the cached price; otherwise, allows the API call and `after_tool_callback` saves the result to the state key.

#### <b>5. Request/Response Modification</b>
* <b><font size='3ptx'>Pattern</font></b>: Alter data just before it's sent to the LLM/tool or just after it's received.
* <b><font size='3ptx'>How</font></b>:
    - <b>`before_model_callback`</b>: Modify `llm_request` (<font color='brown'>e.g., add system instructions based on state</font>).
    - <b>`after_model_callback`</b>: Modify the returned `LlmResponse` (<font color='brown'>e.g., format text, filter content</font>).
    - <b>`before_tool_callback`</b>: Modify the tool `args` dictionary.
    - <b>`after_tool_callback`</b>: Modify the `tool_response` dictionary.
* <b><font size='3ptx'>Example</font></b>: `before_model_callback` appends "`User language preference: Spanish`" to `llm_request.config.system_instruction` if `context.state['lang'] == 'es'`.

#### <b>6. Conditional Skipping of Steps</b>
* <b><font size='3ptx'>Pattern</font></b>: Prevent standard operations (<font color='brown'>agent run, LLM call, tool execution</font>) based on certain conditions.
* <b><font size='3ptx'>How</font></b>: Return a value from a `before_` callback (`Content` from `before_agent_callback`, `LlmResponse` from `before_model_callback`, `dict` from `before_tool_callback`). The framework interprets this returned value as the result for that step, skipping the normal execution.
* <b><font size='3ptx'>Example</font></b>: `before_tool_callback` checks `tool_context.state['api_quota_exceeded']`. If True, it returns `{'error': 'API quota exceeded'}`, preventing the actual tool function from running.

#### <b>7. Tool-Specific Actions (Authentication & Summarization Control)</b>
* <b><font size='3ptx'>Pattern</font></b>: Handle actions specific to the tool lifecycle, primarily authentication and controlling LLM summarization of tool results.
* <b><font size='3ptx'>How</font></b>: Use `ToolContext` within tool callbacks (`before_tool_callback`, `after_tool_callback`).
    - <b>Authentication</b>: Call `tool_context.request_credential(auth_config)` in `before_tool_callback` if credentials are required but not found (<font color='brown'>e.g., via `tool_context.get_auth_response` or state check</font>). This initiates the auth flow.
    - <b>Summarization</b>: Set `tool_context.actions.skip_summarization = True` if the raw dictionary output of the tool should be passed back to the LLM or potentially displayed directly, bypassing the default LLM summarization step.
* <b><font size='3ptx'>Example</font></b>: A `before_tool_callback` for a secure API checks for an auth token in state; if missing, it calls `request_credential`. An `after_tool_callback` for a tool returning structured JSON might set `skip_summarization = True`.

#### <b>8. Artifact Handling</b>
* <b><font size='3ptx'>Pattern</font></b>: Save or load session-related files or large data blobs during the agent lifecycle.
* <b><font size='3ptx'>How</font></b>: Use `callback_context.save_artifact` / `await tool_context.save_artifact` to store data (<font color='brown'>e.g., generated reports, logs, intermediate data</font>). Use `load_artifact` to retrieve previously stored artifacts. Changes are tracked via `Event.actions.artifact_delta`.
* <b><font size='3ptx'>Example</font></b>: An `after_tool_callback` for a "generate_report" tool saves the output file using `await tool_context.save_artifact("report.pdf", report_part)`. A `before_agent_callback` might load a configuration artifact using `callback_context.load_artifact("agent_config.json")`.

### <b><font color='darkgreen'>Best Practices for Callbacks</font></b>
([source](https://google.github.io/adk-docs/callbacks/design-patterns-and-best-practices/#best-practices-for-callbacks))
* <b><font size='3ptx'>Keep Focused</font></b>: Design each callback for a single, well-defined purpose (e.g., just logging, just validation). Avoid monolithic callbacks.
* <b><font size='3ptx'>Mind Performance</font></b>: Callbacks execute synchronously within the agent's processing loop. <font color='darkred'>Avoid long-running or blocking operations</font> (<font color='brown'>network calls, heavy computation</font>). Offload if necessary, but be aware this adds complexity.
* <b><font size='3ptx'>Handle Errors Gracefully</font></b>: Use `try...except/catch` blocks within your callback functions. Log errors appropriately and decide if the agent invocation should halt or attempt recovery. Don't let callback errors crash the entire process.
* <b><font size='3ptx'>Manage State Carefully</font></b>:
    - <b>Be deliberate about reading from and writing to `context.state`</.b>. Changes are immediately visible within the current invocation and persisted at the end of the event processing.
    - <b>Use specific state keys</b> rather than modifying broad structures to avoid unintended side effects.
    - <b>Consider using state prefixes</b> (`State.APP_PREFIX`, `State.USER_PREFIX`, `State.TEMP_PREFIX`) for clarity, especially with persistent `SessionService` implementations.
* <b><font size='3ptx'>Consider Idempotency</font></b>: If a callback performs actions with external side effects (<font color='brown'>e.g., incrementing an external counter</font>), design it to be idempotent (<font color='brown'>safe to run multiple times with the same input</font>) if possible, to handle potential retries in the framework or your application.
* <b><font size='3ptx'>Test Thoroughly</font></b>: Unit test your callback functions using mock context objects. Perform integration tests to ensure callbacks function correctly within the full agent flow.
* <b><font size='3ptx'>Ensure Clarity</font></b>: Use descriptive names for your callback functions. Add clear docstrings explaining their purpose, when they run, and any side effects (<font color='brown'>especially state modifications</font>).
* <b><font size='3ptx'>Use Correct Context Type</font></b>: Always use the specific context type provided (`CallbackContext` for agent/model, `ToolContext` for tools) to ensure access to the appropriate methods and properties.

By applying these patterns and best practices, you can effectively use callbacks to create more robust, observable, and customized agent behaviors in ADK.